## 1.3.2 Communicating with an API

In [ ]:
endpoint = "http://localhost:5000"

# Fill in the correct API key
api_key = "scientist007"

# Create the web API’s URL
authenticated_endpoint = "{}/{}".format(endpoint, api_key)

# Get the web API’s reply to the endpoint
api_response = requests.get(authenticated_endpoint).json()
pprint.pprint(api_response)

# Create the API’s endpoint for the shops
shops_endpoint = "{}/{}/{}/{}".format(endpoint, api_key, "diaper/api/v1.0", "shops")
shops = requests.get(shops_endpoint).json()
print(shops)

# Create the API’s endpoint for items of the shop starting with a "D"
items_of_specific_shop_URL = "{}/{}/{}/{}/{}".format(endpoint, api_key, "diaper/api/v1.0", "items", "DM")
products_of_shop = requests.get(items_of_specific_shop_URL).json()
pprint.pprint(products_of_shop)

Good job! APIs are one way of making data available to you. With the booming Internet of Things (IoT), web APIs have become more prevalent, so it’s essential to know how to interact with them and thus get data into your data lake this way.

## 1.3.3 Streaming records

In [ ]:
# Use the convenience function to query the API
tesco_items = retrieve_products("Tesco")

singer.write_schema(stream_name="products", schema=schema,
                    key_properties=[])

# Write a single record to the stream, that adheres to the schema
singer.write_record(stream_name="products", 
                    record={**tesco_items[0], "store_name": "Tesco"})

for shop in requests.get(SHOPS_URL).json()["shops"]:
    # Write all of the records that you retrieve from the API
    singer.write_records(
      stream_name="products", # Use the same stream name that you used in the schema
      records=({**tesco_items[0], "store_name": shop}
               for item in retrieve_products(shop))
    )    

Using generators like you did in this exercise is a great way of keeping the memory footprint low, which is especially important when you are dealing with a lot of data.

## 1.3.4 Chain taps and targets
#### 输出目标格式的提取的数据 tap =》 extract data

In [ ]:
tap-marketing-api | target-csv --config ingest/data_lake.conf

#### tap什么API | target输出格式 --config config文件所在文件夹位置

## 2.1.1 Read a csv file

In [ ]:
# Read a csv file and set the headers
df = spark.read.options(header = 'true').csv("/home/repl/workspace/mnt/data_lake/landing/ratings.csv")

df.show()

## 2.1.2 Defining a schema


To do this, you use classes from the pyspark.sql.types module. Its StructType() class expects a list of StructField() instances that allow you to add fields to a schema. Various other types, such as ByteType() and IntegerType() are also defined in this module and can be used to specify the data types of each field. 

In [ ]:
# Define the schema
schema = StructType([
  StructField("brand", StringType(), nullable=False),
  StructField("model", StringType(), nullable=False),
  StructField("absorption_rate", ByteType(), nullable=True),
  StructField("comfort", ByteType(), nullable=True)
])

better_df = (spark
             .read
             .options(header="true")
             # Pass the predefined schema to the Reader
             .schema(schema)
             .csv("/home/repl/workspace/mnt/data_lake/landing/ratings.csv"))
pprint(better_df.dtypes)

# 2.2 auto Clean  data

## 2.2.1 Removing invalid rows
Data scientists spend a lot of their time cleaning data, because most data sources they work with are not ready for analytics. Hence, the second step in the data transformation pipeline is cleaning the data.

In [ ]:

# Specify the option to drop invalid rows
ratings = (spark
           .read
           .options(header="True", mode="DROPMALFORMED")
           .csv("/home/repl/workspace/mnt/data_lake/landing/ratings_with_invalid_rows.csv"))
ratings.show()

## 2.2.2 Filling unknown data
The .fillna() method can be used to fill null values for all columns or a subset thereof.

In [ ]:
print("BEFORE")
ratings.show()

print("AFTER")
# Replace nulls with arbitrary value on column subset
ratings = ratings.fillna(4, subset=["comfort"])
ratings.show()

## 2.2.3 Conditionally replacing values
Another common situation is that you have values that you want to replace or that don’t make any sense as we saw in the video. You can select the column to be transformed by using the .withColumn() method, conditionally replace those values using the pyspark.sql.functions.when function when values meet a given condition or leave them unaltered when they don’t with the .otherwise() method.

In [ ]:
from pyspark.sql.functions import col, when

# Add/relabel the column
categorized_ratings = ratings.withColumn(
    "comfort",
    # Express the condition in terms of column operations
    when(col("comfort") > 3, "sufficient").otherwise("insufficient"))

categorized_ratings.show()

# 2.3 transformationb


## 2.3.1 Selecting and renaming columns

In [ ]:
from pyspark.sql.functions import col

# Select the columns and rename the "absorption_rate" column
result = ratings.select([col("brand"),
                       col("model"),
                       col("absorption_rate").alias("absorbency")])

# Show only unique values
result.distinct().show()

## 2.3.2 Grouping and aggregating data

you’ll use the spark.sql aggregation functions avg() to compute the average value of some column in a group, stddev_samp() to compute the standard (sample) deviation and max() (which we alias as sfmax so as not to shadow Python’s built-in max()) to retrieve the largest value of some column in a group.

In [ ]:
from pyspark.sql.functions import col, avg, stddev_samp, max as sfmax

aggregated = (purchased
              # Group rows by 'Country'
              .groupBy(col('Country'))
              .agg(
                # Calculate the average salary per group and rename
                avg('Salary').alias('average_salary'),
                # Calculate the standard deviation per group
                stddev_samp('Salary'),
                # Retain the highest salary per group and rename
                sfmax('Salary').alias('highest_salary')
              )
             )

aggregated.show()

# 2.4 packaging and submit application

编写好的Spark程序一般通过Spark-submit指令的方式提交给Spark集群进行具体的任务计算


## 2.4.1 zip spark file

You archived all of the code in a single zip file, which can be passed to the spark-submit command for distribution in a compute cluster.

In [ ]:
zip --recurse-paths zip_file.zip pipeline_folder

## 2.4.2 Submitting your Spark job

With the dependencies of a job ready to be distributed across a cluster’s nodes, you can submit a job to a cluster easily. In this exercise, you'll be testing the job locally.

spark-submit --py-files PY_FILES MAIN_PYTHON_FILE

with PY_FILES being either a zipped archive, a Python egg or separate Python files that will be placed on the PYTHONPATH environment variable of your cluster's nodes. The MAIN_PYTHON_FILE should be the entry point of your application.

In this particular exercise, the path of the zipped archive is spark_pipelines/pydiaper/pydiaper.zip whereas the path to your application entry point is spark_pipelines/pydiaper/pydiaper/cleaning/clean_ratings.py.

In [ ]:
# shell commond
spark-submit --py-files spark_pipelines/pydiaper/pydiaper.zip spark_pipelines/pydiaper/pydiaper/cleaning/clean_ratings.py.

# 3.2 test 

## 3.2.1 Creating in-memory DataFrames
Creating small datasets for unit tests is an important skill. It improves readability and understanding, because any developer looking at your code, can immediately see the inputs to some function and how they relate to the output. Additionally, you can illustrate how the function behaves with normal data and with exceptional data, like missing or incorrect fields.

In [ ]:
from datetime import date
from pyspark.sql import Row

Record = Row("country", "utm_campaign", "airtime_in_minutes", "start_date", "end_date")

# Create a tuple of records
data = (
  Record("USA", "DiapersFirst", 28, date(2017, 1, 20), date(2017, 1, 27)),
  Record("Germany", "WindelKind", 31, date(2017, 1, 25), None),
  Record("India", "CloseToCloth", 32, date(2017, 1, 25), date(2017, 2, 2))
)

# Create a DataFrame from these records
frame = spark.createDataFrame(data)
frame.show()

   0  1  2
0  1  2  3
1  a  b  c
RangeIndex(start=0, stop=2, step=1)


AttributeError: 'DataFrame' object has no attribute 'row'